# Iniciando com Word2Vec com Gensim!
  
A ideia por trás do Word2Vec é bem simples. Estamos supondo que você pode inferir o significado de uma palavra pela composição que ela mantém. Isso é análogo ao ditado popular "mostre-me seus amigos e eu direi quem você é". Então, se você tem duas palavras que têm vizinhos muito semelhantes (ou seja, o contexto de uso é aproximadamente o mesmo), então essas palavras são provavelmente muito semelhantes em significado ou pelo menos altamente relacionadas. Por exemplo, as palavras em inglês `shocked`,` appalled` e `astonished` são tipicamente usadas em um contexto similar.

A implementação do Gensim é baseada no artigo [original Word2Vec implementation by Google](https://arxiv.org/pdf/1301.3781.pdf) e foi extendido com funcionalidades novas.

### Imports e logging

Primeiro, começamos com os imports e o logging:

In [1]:
# imports necessários para configurar o logging
import gzip
import gensim 
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


### Dataset 
Em seguida, manipularemos nosso conjunto de dados. O segredo para fazer o Word2Vec funcionar é ter muitos dados no formato de texto. Neste caso, uma boa base de dados pode ser encontrada no seguinte link: [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/). Este conjunto de dados tem avaliações completas de usuários de carros e hotéis. Eu concatenei especificamente todas as avaliações de hotéis em um arquivo grande que é de cerca de 97MB comprimido e 229MB não compactado. Vamos usar o arquivo compactado. Cada linha neste arquivo representa uma resenha do hotel. Você pode fazer o download do conjunto de dados do Word2Vec do OpinRank aqui.

Agora, vamos dar uma olhada mais de perto nesses dados, imprimindo a primeira linha. Você pode ver que esta é uma revisão bastante robusto.

In [3]:
data_file = "data/reviews_data.txt.gz"

with gzip.open ('data/reviews_data.txt.gz', 'rb') as f:
    for i,line in enumerate (f):
        print(line)
        break


b"Oct 12 2009 \tNice trendy hotel location not too bad.\tI stayed in this hotel for one night. As this is a fairly new place some of the taxi drivers did not know where it was and/or did not want to drive there. Once I have eventually arrived at the hotel, I was very pleasantly surprised with the decor of the lobby/ground floor area. It was very stylish and modern. I found the reception's staff geeting me with 'Aloha' a bit out of place, but I guess they are briefed to say that to keep up the coroporate image.As I have a Starwood Preferred Guest member, I was given a small gift upon-check in. It was only a couple of fridge magnets in a gift box, but nevertheless a nice gesture.My room was nice and roomy, there are tea and coffee facilities in each room and you get two complimentary bottles of water plus some toiletries by 'bliss'.The location is not great. It is at the last metro stop and you then need to take a taxi, but if you are not planning on going to see the historic sites in Be

### Ler os arquivos dentro de listas
Agora que tivemos uma ideia do nosso conjunto de dados, podemos lê-lo em uma lista para que possamos passar isso para o modelo Word2Vec. Observe no código abaixo, que estou lendo diretamente o arquivo compactado. Também estamos fazendo um pré-processamento moderado das revisões usando `gensim.utils.simple_preprocess (line)`. Isso faz um pré-processamento básico, como tokenização, letras minúsculas, etc, e retorna uma lista de tokens (palavras). 



In [9]:
def read_input(input_file):
    """Esse método lê o arquivo de entrada no formato gzip"""
    
    logging.info("lendo arquivo {0}...esse processo pode demorar".format(input_file))
    
    with gzip.open (input_file, 'rb') as f:
        for i, line in enumerate (f): 

            if (i%10000==0):
                logging.info ("lido {0} revisoes".format (i))
            # feito alguns pre-processamentos e retorna uma lista de palavras
            # para cada revisão no formato texto
            yield gensim.utils.simple_preprocess (line)

# ler uma revisão tokenizada em uma lista
# cada revisão se torna um série de palavras 
# então isso se torna uma lista de listas
documents = list (read_input (data_file))
logging.info ("Terminado de ler os arquivos")    

2019-07-14 16:59:57,634 : INFO : lendo arquivo data/reviews_data.txt.gz...esse processo pode demorar
2019-07-14 16:59:57,636 : INFO : lido 0 revisoes
2019-07-14 16:59:58,963 : INFO : lido 10000 revisoes
2019-07-14 17:00:01,041 : INFO : lido 20000 revisoes
2019-07-14 17:00:02,569 : INFO : lido 30000 revisoes
2019-07-14 17:00:03,980 : INFO : lido 40000 revisoes
2019-07-14 17:00:05,524 : INFO : lido 50000 revisoes
2019-07-14 17:00:07,021 : INFO : lido 60000 revisoes
2019-07-14 17:00:08,291 : INFO : lido 70000 revisoes
2019-07-14 17:00:09,458 : INFO : lido 80000 revisoes
2019-07-14 17:00:10,691 : INFO : lido 90000 revisoes
2019-07-14 17:00:11,872 : INFO : lido 100000 revisoes
2019-07-14 17:00:14,005 : INFO : lido 110000 revisoes
2019-07-14 17:00:15,181 : INFO : lido 120000 revisoes
2019-07-14 17:00:16,379 : INFO : lido 130000 revisoes
2019-07-14 17:00:17,678 : INFO : lido 140000 revisoes
2019-07-14 17:00:18,860 : INFO : lido 150000 revisoes
2019-07-14 17:00:20,077 : INFO : lido 160000 revi

## Treinamento do modelo Word2Vec

Treinar o modelo é bastante simples. Você apenas instancia o Word2Vec e passa os comentários que lemos na etapa anterior (os `documentos`). Então, estamos essencialmente passando uma lista de listas. Onde cada lista na lista principal contém um conjunto de tokens de uma revisão do usuário. O Word2Vec usa todos esses tokens para criar internamente um vocabulário. E por vocabulário, quero dizer um conjunto de palavras únicas.

Depois de construir o vocabulário, só precisamos chamar `train(...)` para começar a treinar o modelo Word2Vec. O treinamento no conjunto de dados [OpinRank] (http://kavita-ganesan.com/entity-ranking-data/) leva cerca de 10 minutos, portanto, seja paciente ao executar seu código neste conjunto de dados.

Nos bastidores, na verdade, estamos treinando uma rede neural simples com uma única camada oculta. Mas, na verdade, não vamos usar a rede neural após o treinamento. Em vez disso, o objetivo é aprender os pesos da camada oculta. Esses pesos são essencialmente os vetores de palavras que estamos tentando aprender.

In [10]:
model = gensim.models.Word2Vec (documents, size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

2019-07-14 17:05:48,958 : WARNING : consider setting layer size to a multiple of 4 for greater performance
2019-07-14 17:05:48,959 : INFO : collecting all words and their counts
2019-07-14 17:05:48,960 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-14 17:05:49,156 : INFO : PROGRESS: at sentence #10000, processed 1655714 words, keeping 25777 word types
2019-07-14 17:05:49,330 : INFO : PROGRESS: at sentence #20000, processed 3317863 words, keeping 35016 word types
2019-07-14 17:05:49,538 : INFO : PROGRESS: at sentence #30000, processed 5264072 words, keeping 47518 word types
2019-07-14 17:05:49,738 : INFO : PROGRESS: at sentence #40000, processed 7081746 words, keeping 56675 word types
2019-07-14 17:05:49,952 : INFO : PROGRESS: at sentence #50000, processed 9089491 words, keeping 63744 word types
2019-07-14 17:05:50,172 : INFO : PROGRESS: at sentence #60000, processed 11013726 words, keeping 76786 word types
2019-07-14 17:05:50,438 : INFO : PROGRESS: a

2019-07-14 17:06:23,287 : INFO : EPOCH 2 - PROGRESS: at 6.21% examples, 950340 words/s, in_qsize 17, out_qsize 2
2019-07-14 17:06:24,292 : INFO : EPOCH 2 - PROGRESS: at 9.56% examples, 991095 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:06:25,292 : INFO : EPOCH 2 - PROGRESS: at 12.40% examples, 1016253 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:06:26,298 : INFO : EPOCH 2 - PROGRESS: at 15.67% examples, 1029617 words/s, in_qsize 20, out_qsize 2
2019-07-14 17:06:27,303 : INFO : EPOCH 2 - PROGRESS: at 18.69% examples, 1043031 words/s, in_qsize 18, out_qsize 1
2019-07-14 17:06:28,308 : INFO : EPOCH 2 - PROGRESS: at 21.79% examples, 1049385 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:06:29,310 : INFO : EPOCH 2 - PROGRESS: at 24.60% examples, 1054682 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:06:30,328 : INFO : EPOCH 2 - PROGRESS: at 28.61% examples, 1055716 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:06:31,351 : INFO : EPOCH 2 - PROGRESS: at 32.59% examples, 1057977 wor

2019-07-14 17:07:22,178 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 17:07:22,186 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 17:07:22,187 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 17:07:22,188 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 17:07:22,201 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 17:07:22,202 : INFO : EPOCH - 3 : training on 41519358 raw words (30345701 effective words) took 29.4s, 1032466 effective words/s
2019-07-14 17:07:23,209 : INFO : EPOCH 4 - PROGRESS: at 3.36% examples, 1044977 words/s, in_qsize 17, out_qsize 2
2019-07-14 17:07:24,210 : INFO : EPOCH 4 - PROGRESS: at 6.94% examples, 1065981 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:07:25,216 : INFO : EPOCH 4 - PROGRESS: at 10.14% examples, 1075075 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:07:26,216 : INFO : EPOCH 4 - PROGRESS: at 13.14

2019-07-14 17:08:19,715 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 17:08:19,717 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 17:08:19,724 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 17:08:19,725 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 17:08:19,733 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 17:08:19,735 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 17:08:19,737 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 17:08:19,752 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 17:08:19,753 : INFO : EPOCH - 5 : training on 41519358 raw words (30350223 effective words) took 29.4s, 1031217 effective words/s
2019-07-14 17:08:19,754 : INFO : training on a 207596790 raw words (151744021 effective words) took 143.7s, 1055904 effective words

2019-07-14 17:09:13,309 : INFO : EPOCH 2 - PROGRESS: at 97.89% examples, 1094631 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:09:13,758 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-07-14 17:09:13,762 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-07-14 17:09:13,772 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 17:09:13,773 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 17:09:13,778 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 17:09:13,786 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 17:09:13,790 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 17:09:13,792 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 17:09:13,792 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 17:09:13,797 : INFO : worker thread finished; awaiting 

2019-07-14 17:10:06,431 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 17:10:06,437 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 17:10:06,438 : INFO : worker thread finished; awaiting finish of 5 more threads
2019-07-14 17:10:06,449 : INFO : worker thread finished; awaiting finish of 4 more threads
2019-07-14 17:10:06,451 : INFO : worker thread finished; awaiting finish of 3 more threads
2019-07-14 17:10:06,454 : INFO : worker thread finished; awaiting finish of 2 more threads
2019-07-14 17:10:06,455 : INFO : worker thread finished; awaiting finish of 1 more threads
2019-07-14 17:10:06,464 : INFO : worker thread finished; awaiting finish of 0 more threads
2019-07-14 17:10:06,465 : INFO : EPOCH - 4 : training on 41519358 raw words (30351907 effective words) took 26.8s, 1134088 effective words/s
2019-07-14 17:10:07,473 : INFO : EPOCH 5 - PROGRESS: at 3.90% examples, 1202361 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:10:

2019-07-14 17:11:03,450 : INFO : EPOCH 6 - PROGRESS: at 86.64% examples, 972370 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:11:04,465 : INFO : EPOCH 6 - PROGRESS: at 89.92% examples, 969902 words/s, in_qsize 16, out_qsize 3
2019-07-14 17:11:05,469 : INFO : EPOCH 6 - PROGRESS: at 92.51% examples, 962063 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:11:06,490 : INFO : EPOCH 6 - PROGRESS: at 95.38% examples, 957473 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:11:07,497 : INFO : EPOCH 6 - PROGRESS: at 98.09% examples, 951973 words/s, in_qsize 20, out_qsize 1
2019-07-14 17:11:08,105 : INFO : worker thread finished; awaiting finish of 9 more threads
2019-07-14 17:11:08,123 : INFO : worker thread finished; awaiting finish of 8 more threads
2019-07-14 17:11:08,127 : INFO : worker thread finished; awaiting finish of 7 more threads
2019-07-14 17:11:08,130 : INFO : worker thread finished; awaiting finish of 6 more threads
2019-07-14 17:11:08,134 : INFO : worker thread finished; awaiting 

2019-07-14 17:11:59,048 : INFO : EPOCH 8 - PROGRESS: at 59.88% examples, 1021087 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:00,050 : INFO : EPOCH 8 - PROGRESS: at 64.02% examples, 1028166 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:01,066 : INFO : EPOCH 8 - PROGRESS: at 67.28% examples, 1024346 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:02,097 : INFO : EPOCH 8 - PROGRESS: at 70.54% examples, 1021406 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:03,101 : INFO : EPOCH 8 - PROGRESS: at 74.47% examples, 1026825 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:04,104 : INFO : EPOCH 8 - PROGRESS: at 77.99% examples, 1031133 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:05,106 : INFO : EPOCH 8 - PROGRESS: at 81.92% examples, 1037907 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:06,115 : INFO : EPOCH 8 - PROGRESS: at 85.94% examples, 1045132 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:07,118 : INFO : EPOCH 8 - PROGRESS: at 90.33% examples, 1052010

2019-07-14 17:12:53,154 : INFO : EPOCH 10 - PROGRESS: at 69.26% examples, 1169196 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:54,165 : INFO : EPOCH 10 - PROGRESS: at 73.39% examples, 1171893 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:55,170 : INFO : EPOCH 10 - PROGRESS: at 77.26% examples, 1173425 words/s, in_qsize 17, out_qsize 2
2019-07-14 17:12:56,177 : INFO : EPOCH 10 - PROGRESS: at 81.22% examples, 1175769 words/s, in_qsize 18, out_qsize 1
2019-07-14 17:12:57,180 : INFO : EPOCH 10 - PROGRESS: at 85.19% examples, 1177402 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:58,188 : INFO : EPOCH 10 - PROGRESS: at 89.57% examples, 1179413 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:12:59,195 : INFO : EPOCH 10 - PROGRESS: at 93.80% examples, 1180985 words/s, in_qsize 19, out_qsize 0
2019-07-14 17:13:00,201 : INFO : EPOCH 10 - PROGRESS: at 97.96% examples, 1181960 words/s, in_qsize 20, out_qsize 0
2019-07-14 17:13:00,625 : INFO : worker thread finished; awaiting finish

(303498829, 415193580)

## Vamos checar o output 
Este primeiro exemplo mostra um caso simples de procurar palavras semelhantes à palavra `dirty`. Tudo o que precisamos fazer aqui é chamar a função `most_similar` e fornecer a palavra` dirty` como o exemplo positivo. Isso retorna o top 10 palavras semelhantes.

In [11]:
w1 = "dirty"
model.wv.most_similar (positive=w1)

2019-07-14 17:15:58,245 : INFO : precomputing L2-norms of word weight vectors


[('filthy', 0.8671504259109497),
 ('stained', 0.7767190933227539),
 ('dusty', 0.7680657505989075),
 ('unclean', 0.7639098167419434),
 ('smelly', 0.7537983655929565),
 ('grubby', 0.7474677562713623),
 ('soiled', 0.7356410026550293),
 ('dingy', 0.718575656414032),
 ('disgusting', 0.7163524031639099),
 ('gross', 0.7155978679656982)]

Isso parece muito bom, certo? Vamos dar uma olhada em mais alguns. Vamos ver a similaridade de `educado`,` frança` e `chocado`. 

In [14]:
# vamos ver as 6 palavras mais similares a 'polite'
w1 = ["polite"]
model.wv.most_similar (positive=w1, topn=6)

[('courteous', 0.9205291271209717),
 ('friendly', 0.8392852544784546),
 ('cordial', 0.8127507567405701),
 ('professional', 0.788678765296936),
 ('attentive', 0.782365083694458),
 ('curteous', 0.7751853466033936)]

In [16]:
# vamos ver as 6 palavras mais similares a 'france'
w1 = ["france"]
model.wv.most_similar(positive=w1, topn=6)

[('canada', 0.6721209287643433),
 ('germany', 0.6644063591957092),
 ('spain', 0.6451458930969238),
 ('mexico', 0.6338759660720825),
 ('hawaii', 0.6228253841400146),
 ('rome', 0.6122733354568481)]

In [17]:
# vamos ver as 6 palavras mais similares a 'france'
w1 = ["shocked"]
model.wv.most_similar (positive=w1,topn=6)

[('horrified', 0.8059837818145752),
 ('amazed', 0.7890470027923584),
 ('appalled', 0.768475353717804),
 ('astonished', 0.7476360201835632),
 ('stunned', 0.735933244228363),
 ('dismayed', 0.7296151518821716)]

Isso é bom. Você pode até especificar vários exemplos positivos para obter coisas que estão relacionadas no contexto fornecido e fornecer exemplos negativos para dizer o que não deve ser considerado como relacionado. No exemplo abaixo, estamos pedindo todos os itens que se referem *cama* (*bed* em inglês):

In [18]:
# palavras relacionadas com cama
w1 = ["bed",'sheet','pillow']
w2 = ['couch']
model.wv.most_similar (positive=w1,negative=w2,topn=10)

[('duvet', 0.718425989151001),
 ('blanket', 0.6985957622528076),
 ('matress', 0.694139838218689),
 ('pillowcase', 0.689792811870575),
 ('mattress', 0.6870702505111694),
 ('quilt', 0.6829710602760315),
 ('pillows', 0.6435014009475708),
 ('foam', 0.6245807409286499),
 ('sheets', 0.6244121789932251),
 ('comforter', 0.5996685028076172)]

### Similaridade entre duas palavras no vocabulário

Você pode até usar o modelo Word2Vec para retornar a semelhança entre duas palavras que estão presentes no vocabulário.

In [22]:
# similaridade de duas palavras diferentes
model.wv.similarity(w1="dirty", w2="smelly")

0.7537983

In [23]:
# similaridades de duas palavras idênticas
model.wv.similarity(w1="dirty", w2="dirty")

1.0

In [24]:
# similaridade de duas palavras opostas
model.wv.similarity(w1="dirty", w2="clean")

0.286456

Debaixo dos panos, os três trechos acima calculam a semelhança de cosseno entre as duas palavras especificadas usando vetores de palavras de cada um. A partir das pontuações, faz sentido que `dirty` seja altamente similar a` smelly`, mas `dirty` é diferente de` clean`. Se você fizer uma semelhança entre duas palavras idênticas, a pontuação será 1.0, já que o intervalo da pontuação de semelhança do cosseno será sempre entre [0.0-1.0]. Você pode ler mais sobre a pontuação de semelhança de cosseno [aqui] (https://en.wikipedia.org/wiki/Cosine_similarity).

### Encontre o estranho
Você pode até usar o Word2Vec para encontrar itens estranhos com uma lista de itens.

In [25]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["cat","dog","france"])

/home/geanderson/anaconda3/lib/python3.7/site-packages/gensim/models/keyedvectors.py:876: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'france'

In [31]:
# Qual dos items abaixo é o estranho da lista?
model.wv.doesnt_match(["bed","pillow","duvet","car"])

'car'

## Entendendo os parâmetros que podem ser utilizados
Para treinar o modelo anteriormente, tivemos que definir alguns parâmetros. Agora, vamos tentar entender o que alguns deles significam. Para referência, este é o comando que usamos para treinar o modelo.

```
model = gensim.models.Word2Vec(documents, size=150, window=10, min_count=2, workers=10)
```

### `size`
O tamanho do vetor denso para representar cada token ou palavra. Se você tiver dados muito limitados, o tamanho deverá ser um valor muito menor. Se você tiver muitos dados, é bom experimentar vários tamanhos. Um valor de 100-150 funcionou bem para o dataset treinado.

### `window`
A distância máxima entre a palavra alvo e a palavra vizinha. Se a posição do seu vizinho for maior que a largura máxima da janela à esquerda e à direita, alguns vizinhos não serão considerados como relacionados à palavra de destino. Em teoria, uma janela menor deve fornecer termos mais relacionados. Se você tiver muitos dados, o tamanho da janela não deve importar muito, desde que seja uma janela de tamanho decente. 

### `min_count`
Contagem de frequência mínima de palavras. O modelo ignoraria as palavras que não statisfy o `min_count`. Palavras extremamente raras geralmente não são importantes, então é melhor livrar-se delas. A menos que seu conjunto de dados seja realmente pequeno, isso não afeta realmente o modelo.

### `workers`
Quantas threads deveríamos usar?

## Quando usar Word2Vec?

Existem muitos cenários de aplicativos para o Word2Vec. Imagine se você precisa construir um léxico de sentimento. Treinar um modelo Word2Vec em grandes quantidades de comentários de usuários ajuda você a conseguir isso. Você tem um léxico não apenas para o sentimento, mas para a maioria das palavras no vocabulário.

Além dos dados de texto bruto não estruturados, você também pode usar o Word2Vec para obter dados mais estruturados. Por exemplo, se você tivesse tags para um milhão de perguntas e respostas do stackoverflow, poderia encontrar tags relacionadas a uma determinada tag e recomendar as relacionadas para exploração. Você pode fazer isso tratando cada conjunto de tags de coexistência como uma "frase" e treinar um modelo Word2Vec nesses dados. Concedido, você ainda precisa de um grande número de exemplos para fazê-lo funcionar.